In [2]:
given = """
1. Too much, in
2. Genetic
3. Sounding weak
4. Very poor
5. Blackjack (hyph.)
6. In a window
7. Volcanic
8. Word after
9. Simon
10. Letters
11. And thin
12. Tall
13. Worker
14. Crossword
15. Sensitive
16. Someone who
17. In the picture
18. Constructor at times
19. Gum flavor
20. Not sedimentary
21. Something in a Playmate’s
22. Sophomore
23. Individual squares
24. View (2 wds)
25. Person
26. Common deli
27. Butler or
28. Profile
29. Red
30. Manufacturer
31. Meat (2 wds)
32. Express
33. Hancock
34. Spanish musical
35. Jupiter
36. Takes care of others
37. Chauffeur
38. Venti
39. Retains its leaves in winter
40. Twinkie
41. Audibly
42. One’s John
43. Not visible
44. Senior
45. Or metamorphic
46. A tree that
47. Instruments
48. Similar
49. With special skill
50. Chewing
51. Flowing material
52. Shell
53. Yellow
54. Materials for
55. Topics (2 wds)
56. Starting a fire
57. Uranus
58. One point of
59. Of the face
60. In nature
61. Less
62. They are in the center
63. It’s purchased at a
64. Available
"""


def parse(s):
  result = []
  for line in s.split('\n'):
    if not line:
      continue
    n, clue = line.split('. ', maxsplit=1)
    n = int(n)
    result.append((n, clue))
  return result

clues = parse(given)

In [7]:
for _, x in clues:
  print(x)

Too much, in
Genetic
Sounding weak
Very poor
Blackjack (hyph.)
In a window
Volcanic
Word after
Simon
Letters
And thin
Tall
Worker
Crossword
Sensitive
Someone who
In the picture
Constructor at times
Gum flavor
Not sedimentary
Something in a Playmate’s
Sophomore
Individual squares
View (2 wds)
Person
Common deli
Butler or
Profile
Red
Manufacturer
Meat (2 wds)
Express
Hancock
Spanish musical
Jupiter
Takes care of others
Chauffeur
Venti
Retains its leaves in winter
Twinkie
Audibly
One’s John
Not visible
Senior
Or metamorphic
A tree that
Instruments
Similar
With special skill
Chewing
Flowing material
Shell
Yellow
Materials for
Topics (2 wds)
Starting a fire
Uranus
One point of
Of the face
In nature
Less
They are in the center
It’s purchased at a
Available


In [3]:
targets = [
  33, 73, 65, 12, 70, 64, 85, 17,
  81,
  52, 14,
  49, 82, 41,
  60,
  121, 69, 57,
  6, 110, 29,
  25, 125, 32,
  58,
  108, 75, 115,
  62, 46,
]

In [8]:
for i in sorted(targets):
  print(i)

6
12
14
17
25
29
32
33
41
46
49
52
57
58
60
62
64
65
69
70
73
75
81
82
85
108
110
115
121
125


In [4]:
import forge
from data import warehouse
from puzzle.puzzlepedia import prod_config

prod_config.init()

In [5]:
import itertools
import re

_END = r'[^\w](\w+)$'
_BAD_ENDINGS = {'s', 'be', 'of', 'in', 'the', 'has', 'for', 'to'}

def process(clues, targets):
  bigrams = warehouse.get('/words/bigram')
  candidates = set(targets)
  results = []
  for (i1, c1), (i2, c2) in itertools.combinations(clues, 2):
    s = i1 + i2
    if s not in targets:
      continue
    best_weight = -1
    first, second = c1, c2
    for a, b in [(c1, c2), (c2, c1)]:
      b_ending = re.findall(_END, b)
      bad_start = '(' in a
      if bad_start or b_ending and b_ending[0] in _BAD_ENDINGS:
        weight = -1
      else:
        end = a.lower().split()[-1]  # Join end of first...
        start = b.lower().split()[0]  # ...with start of last.
        weight = bigrams.get('%s %s' % (end, start), 0)
      if weight > best_weight:
        best_weight = weight
        first, second = a, b
    if best_weight >= 0:
      results.append((best_weight, '%s\t%s\t%s\t%s|%s' % (s, i1, i2, first, second)))
  return sorted(results, key=lambda x: x[0], reverse=True)

In [6]:
for weight, row in process(clues, set(targets)):
  print(row)

108	46	62	A tree that|They are in the center
73	27	46	Butler or|A tree that
70	27	43	Butler or|Not visible
33	6	27	Butler or|In a window
70	6	64	Available|In a window
81	17	64	Available|In the picture
52	6	46	A tree that|In a window
12	1	11	Too much, in|And thin
60	6	54	Materials for|In a window
121	59	62	They are in the center|Of the face
46	1	45	Too much, in|Or metamorphic
82	24	58	One point of|View (2 wds)
115	54	61	Materials for|Less
65	11	54	Materials for|And thin
81	23	58	One point of|Individual squares
85	25	60	Person|In nature
69	11	58	One point of|And thin
29	11	18	Constructor at times|And thin
70	25	45	Person|Or metamorphic
64	18	46	Constructor at times|A tree that
70	16	54	Materials for|Someone who
75	27	48	Butler or|Similar
62	1	61	Too much, in|Less
64	6	58	One point of|In a window
75	17	58	One point of|In the picture
62	11	51	Flowing material|And thin
57	11	46	A tree that|And thin
75	11	64	Available|And thin
70	8	62	Word after|They are in the center
70	11	59	Of the face|An